In [1]:
import requests
import pandas
from arcgis.features import GeoAccessor

In [2]:
stem_path = r'C:/Users/tjjoh/Documents/GIS5571'

gdb_path = stem_path + '/FinalProject_GDB.gdb'
noaa_fc_path = gdb_path + r'/NOAA'

idw1_noaa_path = gdb_path + r'/NOAA_IDW1'
idw2_noaa_path = gdb_path + r'/NOAA_IDW2'
natneigh_noaa_path = gdb_path + r'/NOAA_natneigh'
spline_noaa_path = gdb_path + r'/NOAA_spline'
kriging_noaa_path = gdb_path + r'/NOAA_kriging'

In [3]:
#URL for most recent data
url = r'https://api.synopticlabs.org/v2/stations/latest?token=d8c6aee36a994f90857925cea26934be&bbox=-105%2C42%2C-86.5%2C49.5&minmax=1&minmaxtype=local&units=temp%7Cf%2Cspeed%7Cmph&within=90&status=active&qc_checks=all'

In [4]:
#URL for select time
url = r'https://api.synopticlabs.org/v2/stations/nearesttime?token=d8c6aee36a994f90857925cea26934be&bbox=-105%2C42%2C-86.5%2C49.5&minmax=1&minmaxtype=local&units=temp%7Cf%2Cspeed%7Cmph&attime=202403251800&within=90&status=active&qc_checks=all'

In [5]:
response = requests.get(url)

In [6]:
noaa_df = pandas.json_normalize(response.json()['STATION'])

In [7]:
noaa_df = noaa_df.rename(columns = {'OBSERVATIONS.air_temp_value_1.value' : 'Temperature',
                                    'OBSERVATIONS.relative_humidity_value_1.value' : 'Humidity',
                                    'OBSERVATIONS.wind_speed_value_1.value' : 'WindSpeed',
                                    'OBSERVATIONS.wind_direction_value_1.value' : 'WindDirection',
                                    'OBSERVATIONS.altimeter_value_1.value' : 'Pressure',
                                   })

In [8]:
noaa_df = noaa_df[['ID','NAME','ELEVATION','LATITUDE','LONGITUDE','STATE','TIMEZONE',
                   'Temperature','Humidity','WindSpeed','WindDirection','Pressure']]

In [9]:
noaa_sedf = GeoAccessor.from_xy(noaa_df, x_column = 'LONGITUDE', y_column = 'LATITUDE', sr = 4326)

In [10]:
noaa_fc = noaa_sedf.spatial.to_featureclass(location = noaa_fc_path)

In [11]:
#First-order inverse distance weighting interpolation
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    IDW1 = arcpy.sa.Idw(
        in_point_features = noaa_fc_path,
        z_field = 'Temperature',
        cell_size = 0.021010908,
        power = 1,
        search_radius = 'VARIABLE 12',
        in_barrier_polyline_features = None
    )
    IDW1.save(idw1_noaa_path)

In [12]:
#Second-order inverse distance weighting interpolation
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    IDW2 = arcpy.sa.Idw(
        in_point_features = noaa_fc_path,
        z_field = 'Temperature',
        cell_size = 0.021010908,
        power = 2,
        search_radius = 'VARIABLE 12',
        in_barrier_polyline_features = None
    )
    IDW2.save(idw2_noaa_path)

In [13]:
#Natural neighbor interpolation
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    NatNeigh = arcpy.sa.NaturalNeighbor(
        in_point_features = noaa_fc_path,
        z_field = 'Temperature',
        cell_size = 0.021010908
    )
    NatNeigh.save(natneigh_noaa_path)

In [14]:
#Spline interpolation
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    Spline = arcpy.sa.Spline(
        in_point_features = noaa_fc_path,
        z_field = 'Temperature',
        cell_size = 0.021010908,
        spline_type = 'REGULARIZED',
        weight = 0.1,
        number_points = 12
    )
    Spline.save(spline_noaa_path)

In [15]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    Krig = arcpy.sa.Kriging(
        in_point_features = noaa_fc_path,
        z_field = 'Temperature',
        kriging_model = 'Spherical # # # #',
        cell_size = 0.021010908,
        search_radius = 'VARIABLE 12',
        out_variance_prediction_raster = None
    )
    Krig.save(kriging_noaa_path)